# NativeClient sample notebook
This notebook is an example notebook that explains how to use the AutoML NativeClient to train a machine learning model.

In [ ]:
import sys
import os
import argparse
import logging
import uuid
import time
from datetime import datetime

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from automl.client.core.nativeclient import AutoMLClassifier
from automl.client.core.nativeclient import AutoMLNativeClient
from azureml.core.authentication import AbstractAuthentication

# Uncomment the below line to disable HTTPS verification (in case you need to debug the API calls)
# %env CURL_CA_BUNDLE=

# Setting up inputs
### Authentication
AzureML's workspace objects are used for authentication and service discovery.
Make sure to change the required parameters below.

In [ ]:
from azureml.core import Workspace

# Create the workspace using the specified parameters
workspace = Workspace.create(name = 'workspace name',
                             subscription_id = 'subscription id',
                             resource_group = 'resource group name', 
                             location = 'region',
                             create_resource_group = True,
                             sku = 'basic',
                             exist_ok = True)


### Data preparation

In [ ]:
from sklearn.datasets import load_iris

# Load the dataset. X (features) can be a numpy array or pandas dataframe and y (labels) should be a numpy array.
data = load_iris()
X = data.data
y = data.target

# Running AutoML Classification
This test code only does simple splitting, training and scoring. Model explanations are ignored for now.
Regression is very similar to the classification case, the only differences in usage are that you will use `AutoMLRegressor` and a different metric type.

In [ ]:
from azureml.automl.core.shared import constants

# You should use one of the metric types printed out here during classifier setup.
print('Metric types available for classification:')
print(constants.Metric.CLASSIFICATION_PRIMARY_SET)
print('Metric types available for regression:')
print(constants.Metric.REGRESSION_PRIMARY_SET)

# Instantiate the AutoMLNativeClient. You must pass in an azureml.core.Workspace object.
client = AutoMLNativeClient(
    workspace=workspace
)

# Here, you can also choose to split the dataset into training/validation sets manually instead of using cross validation,
# in which case you would omit n_cross_validations and specify X_valid and y_valid instead.
# If you want to perform a regression experiment, just substitute AutoMLClassifier with AutoMLRegressor (make sure to update
# the primary metric as well).
automl_classifier = AutoMLClassifier(
    client=client,
    name='AutoMLNativeClient_SampleNotebook_{}'.format(time.time()).replace('.', ''),
    debug_log='automl_errors.log',
    primary_metric=constants.Metric.AUCWeighted,
    iterations=3,
    preprocess=True,
    blacklist_models=['KNeighborsClassifier', 'SVCWrapper'],
    iteration_timeout_minutes=5,
    enable_voting_ensemble=False,
    n_cross_validations=10
)

# Train the model.
local_run = automl_classifier.fit(
    X=X,
    y=y,
    compute_target='local',
    show_output=True
)

# Retrieve the best run as well as the corresponding model.
best_run, fitted_model = local_run.get_output(metric=constants.Metric.AUCWeighted)

In [ ]:
# Check model accuracy
y_pred = fitted_model.predict(X_test)
acc = sum(y_pred == y_test) / len(y_pred)

print(acc)